In [27]:
from IPython.display import clear_output
import numpy as np
import time
import random

In [28]:
class Q_Learning:
    def __init__(self, size, episodes, epsilon, discount_factor):
        self.size = size
        self.agent = None        
        self.wumpus = None
        self.gold = None
        self.pits = None
        self.episodes = episodes
        self.epsilon = epsilon
        self.discount_factor = discount_factor
        self.actions = ["up", "right", "down", "left"]
        self.set_coords()
        self.set_rewards()

    def set_coords(self): #Escolhe coordenadas aleatórias para os itens do mundo
        # Distribui poços aleatoriamente
        num_pits = random.randint(1, self.size) #Define a qtd de poços aleatoriamente
        self.pits = []
        for _ in range(num_pits): #Coordenadas aleatorias pra cada poço
            self.pits.append((random.randint(0, self.size - 1),random.randint(0, self.size - 1)))

        # Distribui o Wumpus aleatoriamente
        self.wumpus = (random.randint(0, self.size - 1),random.randint(0, self.size - 1))
        # Distribui o Ouro aleatoriamente
        self.gold = (random.randint(0, self.size - 1),random.randint(0, self.size - 1))         
        
        self.agent = (0,0) # Colocar o agente na posição inicial

    def set_rewards(self):
        # Usando numpy pra criar matrizes facilmente
        # Matriz para armazenar as recompensas pra cada posicao
        self.rewards = np.full((self.size, self.size), -1) #Tamanho de Size, iniciada com -1 de Reward
        for pit in self.pits: # Atribuindo -100 de recompensa aos poços
            self.rewards[pit] = -100

        self.rewards[self.wumpus] = -150 # Atribuindo -150 de recompensa ao Wumpus
        self.rewards[self.gold] = 200 # Atribuindo 200 de recompensa ao Ouro

        # Matriz 3D para armazenar os valores Q para cada par estado-ação
        self.q_values = np.zeros((self.size, self.size, 4)) #Iniciado com 0
        # print(np.shape(self.rewards))
        # print(np.shape(self.q_values))

    def __str__(self):
        output = "  |0|1|2|3|\n"
        for row in range(self.size):
            output += "|%d"%(row)
            for col in range(self.size):
                if(row == self.agent[0] and col == self.agent[1]):
                    output += "|A"
                elif(self.rewards[row][col] == -1):
                    output += "| "
                elif(self.rewards[row][col] == -100):
                    output += "|P"                    
                elif(self.rewards[row][col] == -150):
                    output += "|W"                    
                elif(self.rewards[row][col] == 200):
                    output += "|G"
            output += "|\n"
        return output
               
    #Retorna True, se o estado analisado não for Poço, Wumpus ou Ouro
    def is_terminal_state(self, row, col):
        if self.rewards[row, col] == -1:
            return False
        else:
            return True

    #Algoritmo epsilon-greedy.
    #Usando epsilon como indicador de probabilidade, retorna se uma ação de explotation ou exploration deve ser feita
    def get_next_action(self, cur_row_index, cur_col_index, epsilon):
        #Se um valor aleatório entre 0 e 1 for menor que epsilon,
        if np.random.random() < epsilon:
            #Escolha o "melhor" valor da tabela Q para esse estado
            return np.argmax(self.q_values[cur_row_index, cur_col_index])
        else:  #Se não, escolha uma ação aleatória
            return np.random.randint(4)

    # define uma função que obterá a próxima localização com base na ação escolhida
    def get_next_location(self, cur_row_index, cur_col_index, action_index):
        new_row_index = cur_row_index  # Mantém a linha atual como padrão
        new_col_index = cur_col_index  # Mantém a coluna atual como padrão
        
        # Se a ação for 'up' e não estiver na primeira linha
        if self.actions[action_index] == 'up' and cur_row_index > 0:  
            new_row_index -= 1  # Decrementa o índice da linha para mover para cima
        
        # Se a ação for 'right' e não estiver na última coluna
        elif self.actions[action_index] == 'right' and cur_col_index < self.size - 1:  
            new_col_index += 1  # Incrementa o índice da coluna para mover para a direita
        
        # Se a ação for 'down' e não estiver na última linha
        elif self.actions[action_index] == 'down' and cur_row_index < self.size - 1:  
            new_row_index += 1  # Incrementa o índice da linha para mover para baixo
        
        # Se a ação for 'left' e não estiver na primeira coluna
        elif self.actions[action_index] == 'left' and cur_col_index > 0:  
            new_col_index -= 1  # Decrementa o índice da coluna para mover para a esquerda

        return new_row_index, new_col_index  # Retorna a nova linha e coluna da próxima localização

    def train(self):
        for episode in range(self.episodes):
            row_index, col_index = [0, 0]  #Define a posição inicial do agente como [0, 0]

            #Enquanto a posição atual não for um estado terminal
            while not self.is_terminal_state(row_index, col_index):  
                # Obtém a próxima ação a ser executada
                action_index = self.get_next_action(row_index, col_index, self.epsilon)  
                #Armazena a posição anterior do agente
                old_row_index, old_col_index = row_index, col_index  
                # Calcula o próxima passo do agente e põe o agente lá
                row_index, col_index = self.get_next_location(row_index, col_index, action_index)  
                self.agent = (row_index, col_index) #Atualiza a posição do agente
                #Obtém a recompensa da nova localização
                reward = self.rewards[row_index, col_index]
                
                # Calcula o novo valor Q, pela formula tradicional de Q-learning
                new_q_value = reward + (self.discount_factor * np.max(self.q_values[row_index, col_index]))  

                self.q_values[old_row_index, old_col_index, action_index] = new_q_value #Atualiza o valor Q da ação tomada
          

    def get_path(self, verbose=False):
        start_row_index, start_col_index = [0, 0]  #Define a posição inicial como [0, 0]
        
        #Se a posição inicial for um estado terminal
        if self.is_terminal_state(start_row_index, start_col_index):  
            return []  # Retorna uma lista vazia, já que não há caminho a ser percorrido
        else:
            cur_row_index, cur_col_index = start_row_index, start_col_index #Define a posição atual como a posição inicial
            path = []  # Lista para armazenar o caminho percorrido
            path.append([cur_row_index, cur_col_index])  # Adiciona a posição inicial ao caminho

            # Enquanto a posição atual não for um estado terminal
            while not self.is_terminal_state(cur_row_index, cur_col_index):  
                action_index = self.get_next_action(cur_row_index, cur_col_index, self.epsilon)  # Obtém a próxima ação a ser executada
                cur_row_index, cur_col_index = self.get_next_location(cur_row_index, cur_col_index, action_index)  # Calcula a próxima localização
                path.append([cur_row_index, cur_col_index])  # Adiciona a nova posição ao caminho percorrido
                self.agent = (cur_row_index, cur_col_index)  # Atualiza a posição do agente

                if verbose:  # Se o modo verbose estiver ativado
                    print(self)  # Imprime o ambiente (representação do mundo) atual
                    clear_output(wait=True)  # Limpa a saída do console
                    time.sleep(0.45)  # Aguarda um curto período de tempo para visualização mais clara

            if verbose:
                print(path)  # Imprime o caminho percorrido se o modo verbose estiver ativado
            return path  # Retorna o caminho percorrido


Epsilon:<br>
O parâmetro epsilon (ε) no algoritmo epsilon-greedy controla a proporção de exploração e explotação.<br>
Ele determina a probabilidade de escolher uma ação de exploração (aleatória) em vez de uma ação de explotação (com base nas estimativas atuais de valor).

Discount Factor:<br>
O fator de desconto é um valor no intervalo [0, 1] que indica o quão "importante" é uma recompensa futura em comparação com uma recompensa imediata.<br>
Um valor de discount_factor próximo de 0 indica que o agente dá mais peso às recompensas imediatas, enquanto um valor próximo de 1 indica que o agente considera recompensas futuras igualmente ou até mais importantes do que recompensas imediatas.

In [29]:
mundo_wumpus = Q_Learning(size = 4, episodes = 10000, epsilon = 0.6, discount_factor = 0.8)
mundo_wumpus.train()
mundo_wumpus.get_path(verbose=True)
print(mundo_wumpus)

[[0, 0], [0, 0], [0, 1], [1, 1], [1, 2], [2, 2], [1, 2], [1, 3]]
  |0|1|2|3|
|0| | |W| |
|1| | | |A|
|2| |P| | |
|3| | | | |

